<a href="https://colab.research.google.com/github/casangi/cngi_prototype/blob/master/docs/imaging/mosaic_image_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mosaic Imaging

This notebook will demonstrate how to create a joint mosaic image. Data is taken from all the pointings in an input visibility dataset and combined to have a single phase-reference center. 

This walkthrough is designed to be run in a Jupyter notebook on Google Colaboratory. To open the notebook in colab, go [here](https://colab.research.google.com/github/casangi/cngi_prototype/blob/master/docs/imaging/mosaic_image_example.ipynb).

## Installation


In [1]:
import os
os.system("pip install --extra-index-url https://test.pypi.org/simple/ cngi-prototype==0.0.70")
print('complete') 

complete


## Dataset

The simulated dataset consists of three fields which contain four point sources over three frequency channels. The ALMA layout ```alma.cycle6.3.cfg``` is used, which can be found [here](https://open-bitbucket.nrao.edu/projects/CASA/repos/casa-data/browse/alma/simmos). 

In [69]:
#alma12m_3field_dovpTrue.vis.zarr.zip https://drive.google.com/file/d/1KzWk0Xg8-xpljTL6m8WEE8KbfLTpskRg/view?usp=sharing                                        
id = '1KzWk0Xg8-xpljTL6m8WEE8KbfLTpskRg'
os.system('curl -c ./cookie -s -L "https://drive.google.com/uc?export=download&id=%s"' % id)
os.system('curl -Lb ./cookie "https://drive.google.com/uc?export=download&confirm=`awk \'/download/ {print $NF}\' ./cookie`&id=%s" -o vis.zarr.zip' % id)
os.system('unzip vis.zarr.zip')


#alma12m_3field_dovpTrue_gridder_mosaic.img.zarr.zip https://drive.google.com/file/d/1YdJGBi2qtdCuJ6dm4xrXU5zvXeJc7w3v/view?usp=sharing
id = '1YdJGBi2qtdCuJ6dm4xrXU5zvXeJc7w3v'
os.system('curl -c ./cookie -s -L "https://drive.google.com/uc?export=download&id=%s"' % id)
os.system('curl -Lb ./cookie "https://drive.google.com/uc?export=download&confirm=`awk \'/download/ {print $NF}\' ./cookie`&id=%s" -o vis.zarr.zip' % id)
os.system('unzip vis.zarr.zip')

print('complete') 

complete


## Load Dataset

In [1]:
import xarray as xr
from cngi.dio import read_vis

xr.set_options(display_style="html")

infile = "alma12m_3field_dovpTrue.vis.zarr"
mxds = read_vis(infile)
mxds.xds0

<xarray.Dataset>
Dimensions:         (baseline: 903, chan: 3, pol: 1, pol_id: 1, spw_id: 1, time: 192, uvw_index: 3)
Coordinates:
  * baseline        (baseline) int64 0 1 2 3 4 5 6 ... 897 898 899 900 901 902
  * chan            (chan) float64 3.4e+11 3.74e+11 4.08e+11
    chan_width      (chan) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    effective_bw    (chan) float64 dask.array<chunksize=(1,), meta=np.ndarray>
  * pol             (pol) int32 9
  * pol_id          (pol_id) int32 0
    resolution      (chan) float64 dask.array<chunksize=(1,), meta=np.ndarray>
  * spw_id          (spw_id) int32 0
  * time            (time) datetime64[ns] 2011-05-27T04:32:14.772603989 ... 2...
Dimensions without coordinates: uvw_index
Data variables:
    ANTENNA1        (time, baseline) int32 dask.array<chunksize=(32, 301), meta=np.ndarray>
    ANTENNA2        (time, baseline) int32 dask.array<chunksize=(32, 301), meta=np.ndarray>
    ARRAY_ID        (time, baseline) int32 dask.array<chunksize=(32, 301), meta=np.ndarray>
    CORRECTED_DATA  (time, baseline, chan, pol) complex128 dask.array<chunksize=(32, 301, 1, 1), meta=np.ndarray>
    DATA            (time, baseline, chan, pol) complex128 dask.array<chunksize=(32, 301, 1, 1), meta=np.ndarray>
    EXPOSURE        (time, baseline) float64 dask.array<chunksize=(32, 301), meta=np.ndarray>
    FEED1           (time, baseline) int32 dask.array<chunksize=(32, 301), meta=np.ndarray>
    FEED2           (time, baseline) int32 dask.array<chunksize=(32, 301), meta=np.ndarray>
    FIELD_ID        (time, baseline) int32 dask.array<chunksize=(32, 301), meta=np.ndarray>
    FLAG            (time, baseline, chan, pol) bool dask.array<chunksize=(32, 301, 1, 1), meta=np.ndarray>
    INTERVAL        (time, baseline) float64 dask.array<chunksize=(32, 301), meta=np.ndarray>
    MODEL_DATA      (time, baseline, chan, pol) complex128 dask.array<chunksize=(32, 301, 1, 1), meta=np.ndarray>
    OBSERVATION_ID  (time, baseline) int32 dask.array<chunksize=(32, 301), meta=np.ndarray>
    PROCESSOR_ID    (time, baseline) int32 dask.array<chunksize=(32, 301), meta=np.ndarray>
    SCAN_NUMBER     (time, baseline) int32 dask.array<chunksize=(32, 301), meta=np.ndarray>
    STATE_ID        (time, baseline) int32 dask.array<chunksize=(32, 301), meta=np.ndarray>
    TIME_CENTROID   (time, baseline) float64 dask.array<chunksize=(32, 301), meta=np.ndarray>
    UVW             (time, baseline, uvw_index) float64 dask.array<chunksize=(32, 301, 3), meta=np.ndarray>
    WEIGHT          (time, baseline, chan, pol) float64 dask.array<chunksize=(32, 301, 1, 1), meta=np.ndarray>
Attributes:
    corr_product:     [[0, 0]]
    data_groups:      [{'1': {'data': 'DATA', 'flag': 'FLAG', 'id': '1', 'uvw...
    freq_group:       0
    freq_group_name:  Group 1
    if_conv_chain:    0
    meas_freq_ref:    1
    name:             Band_8
    net_sideband:     1
    num_chan:         3
    num_corr:         1
    ref_frequency:    340000000000.0
    total_bandwidth:  101999999999.99998

## Grid Parameters

In [2]:
grid_parms = {}
grid_parms['chan_mode'] = 'cube'
grid_parms['image_size'] = [1000,720]
grid_parms['cell_size'] = [0.04,0.04]
grid_parms['fft_padding'] = 1.0
grid_parms['phase_center'] = mxds.FIELD.PHASE_DIR[1,0,:].data.compute()

## Direction Rotation

The UVW coordinates must be rotated and the visbility DATA must be phase rotated, relative to the mosaic phase center specified by ```rotation_parms['image_phase_center']```. 

[direction_rotate documentation](https://cngi-prototype.readthedocs.io/en/latest/_api/autoapi/ngcasa/imaging/make_psf/index.html#ngcasa.imaging.direction_rotate.direction_rotate)

In [4]:
from ngcasa.imaging import direction_rotate
import numpy as np
import dask
 
xr.set_options(display_style="html")

infile = "alma12m_3field_dovpTrue.vis.zarr"
mxds = read_vis(infile,chunks={'time':192,'baseline':903,'chan':3})

sel_parms = {}
sel_parms['xds'] = 'xds0'
sel_parms['data_group_in_id'] = 1

rotation_parms = {}
rotation_parms['new_phase_center'] = grid_parms['phase_center']
rotation_parms['common_tangent_reprojection'] = True

mxds = direction_rotate(mxds, rotation_parms, sel_parms)

mxds.xds0

######################### Start direction_rotate #########################
Setting default single_precision  to  True
Setting data_group_in  to  {'data': 'DATA', 'flag': 'FLAG', 'id': '1', 'uvw': 'UVW', 'weight': 'WEIGHT'}
Setting default data_group_out  to  {'data': 'DATA_ROT', 'flag': 'FLAG', 'id': '3', 'uvw': 'UVW_ROT', 'weight': 'WEIGHT'}
######################### Created graph for direction_rotate #########################


<xarray.Dataset>
Dimensions:         (baseline: 903, chan: 3, pol: 1, pol_id: 1, spw_id: 1, time: 192, uvw_index: 3)
Coordinates:
  * baseline        (baseline) int64 0 1 2 3 4 5 6 ... 897 898 899 900 901 902
  * chan            (chan) float64 3.4e+11 3.74e+11 4.08e+11
  * pol             (pol) int64 9
  * time            (time) datetime64[ns] 2011-05-27T04:32:14.772603989 ... 2...
    chan_width      (chan) float64 dask.array<chunksize=(3,), meta=np.ndarray>
    effective_bw    (chan) float64 dask.array<chunksize=(3,), meta=np.ndarray>
  * pol_id          (pol_id) int32 0
    resolution      (chan) float64 dask.array<chunksize=(3,), meta=np.ndarray>
  * spw_id          (spw_id) int32 0
Dimensions without coordinates: uvw_index
Data variables:
    ANTENNA1        (time, baseline) int32 dask.array<chunksize=(192, 903), meta=np.ndarray>
    ANTENNA2        (time, baseline) int32 dask.array<chunksize=(192, 903), meta=np.ndarray>
    ARRAY_ID        (time, baseline) int32 dask.array<chunksize=(192, 903), meta=np.ndarray>
    CORRECTED_DATA  (time, baseline, chan, pol) complex128 dask.array<chunksize=(192, 903, 3, 1), meta=np.ndarray>
    DATA            (time, baseline, chan, pol) complex128 dask.array<chunksize=(192, 903, 3, 1), meta=np.ndarray>
    EXPOSURE        (time, baseline) float64 dask.array<chunksize=(192, 903), meta=np.ndarray>
    FEED1           (time, baseline) int32 dask.array<chunksize=(192, 903), meta=np.ndarray>
    FEED2           (time, baseline) int32 dask.array<chunksize=(192, 903), meta=np.ndarray>
    FIELD_ID        (time, baseline) int32 dask.array<chunksize=(192, 903), meta=np.ndarray>
    FLAG            (time, baseline, chan, pol) bool dask.array<chunksize=(192, 903, 3, 1), meta=np.ndarray>
    INTERVAL        (time, baseline) float64 dask.array<chunksize=(192, 903), meta=np.ndarray>
    MODEL_DATA      (time, baseline, chan, pol) complex128 dask.array<chunksize=(192, 903, 3, 1), meta=np.ndarray>
    OBSERVATION_ID  (time, baseline) int32 dask.array<chunksize=(192, 903), meta=np.ndarray>
    PROCESSOR_ID    (time, baseline) int32 dask.array<chunksize=(192, 903), meta=np.ndarray>
    SCAN_NUMBER     (time, baseline) int32 dask.array<chunksize=(192, 903), meta=np.ndarray>
    STATE_ID        (time, baseline) int32 dask.array<chunksize=(192, 903), meta=np.ndarray>
    TIME_CENTROID   (time, baseline) float64 dask.array<chunksize=(192, 903), meta=np.ndarray>
    UVW             (time, baseline, uvw_index) float64 dask.array<chunksize=(192, 903, 3), meta=np.ndarray>
    WEIGHT          (time, baseline, chan, pol) float64 dask.array<chunksize=(192, 903, 3, 1), meta=np.ndarray>
    UVW_ROT         (time, baseline, uvw_index) float64 dask.array<chunksize=(192, 903, 3), meta=np.ndarray>
    DATA_ROT        (time, baseline, chan, pol) complex128 dask.array<chunksize=(192, 903, 3, 1), meta=np.ndarray>
Attributes:
    corr_product:     [[0, 0]]
    data_groups:      [{'1': {'data': 'DATA', 'flag': 'FLAG', 'id': '1', 'uvw...
    freq_group:       0
    freq_group_name:  Group 1
    if_conv_chain:    0
    meas_freq_ref:    1
    name:             Band_8
    net_sideband:     1
    num_chan:         3
    num_corr:         1
    ref_frequency:    340000000000.0
    total_bandwidth:  101999999999.99998

## Make Imaging Weights

[make_imaging_weight documentation](https://cngi-prototype.readthedocs.io/en/latest/_api/autoapi/ngcasa/imaging/make_imaging_weight/index.html#ngcasa.imaging.make_imaging_weight.make_imaging_weight)

In [5]:
from ngcasa.imaging import make_imaging_weight

imaging_weights_parms = {}
imaging_weights_parms['weighting'] = 'natural'

sel_parms = {}
sel_parms['xds'] = 'xds0'
sel_parms['data_group_in_id'] = 3

mxds = make_imaging_weight(mxds, imaging_weights_parms, grid_parms, sel_parms)

######################### Start make_imaging_weights #########################
Setting data_group_in  to  {'data': 'DATA_ROT', 'flag': 'FLAG', 'id': '3', 'uvw': 'UVW_ROT', 'weight': 'WEIGHT'}
Setting default data_group_out  to  {'data': 'DATA_ROT', 'flag': 'FLAG', 'id': '3', 'uvw': 'UVW_ROT', 'weight': 'WEIGHT', 'imaging_weight': 'IMAGING_WEIGHT'}
Since weighting is natural input weight will be reused as imaging weight.
######################### Created graph for make_imaging_weight #########################
[{'1': {'data': 'DATA', 'flag': 'FLAG', 'id': '1', 'uvw': 'UVW', 'weight': 'WEIGHT'}, '2': {'data': 'CORRECTED_DATA', 'flag': 'FLAG', 'id': '2', 'uvw': 'UVW', 'weight': 'WEIGHT'}, '3': {'data': 'DATA_ROT', 'flag': 'FLAG', 'id': '3', 'uvw': 'UVW_ROT', 'weight': 'WEIGHT', 'imaging_weight': 'WEIGHT'}}]


## Make Gridding Convolution Functions

[make_gridding_convolution_function](https://cngi-prototype.readthedocs.io/en/latest/_api/autoapi/ngcasa/imaging/make_gridding_convolution_function/index.html#ngcasa.imaging.make_gridding_convolution_function.make_gridding_convolution_function)



In [9]:
from ngcasa.imaging import make_gridding_convolution_function
import numpy as np
import dask.array as da

gcf_parms = {}
gcf_parms['function'] = 'alma_airy'
gcf_parms['list_dish_diameters'] = np.array([10.7])
gcf_parms['list_blockage_diameters'] = np.array([0.75])

mxds = mxds.assign_coords(beam_ids=xr.DataArray(da.from_array([6]), dims=('beam')))
beam_id_feed = np.zeros((len(mxds.antenna_ids)),dtype=np.int32)+6
mxds.FEED['beam_id'] = xr.DataArray(da.from_array(beam_id_feed), dims=('d0'))

unique_ant_indx = mxds.ANTENNA.DISH_DIAMETER.values
unique_ant_indx[unique_ant_indx == 12.0] = 0

gcf_parms['unique_ant_indx'] = unique_ant_indx.astype(int)
gcf_parms['phase_center'] = grid_parms['phase_center']

sel_parms = {}
sel_parms['xds'] = 'xds0'
sel_parms['data_group_in_id'] = 3

gcf_xds = make_gridding_convolution_function(mxds, gcf_parms, grid_parms, sel_parms)

######################### Start make_gridding_convolution_function #########################
Setting data_group_in  to  {'data': 'DATA_ROT', 'flag': 'FLAG', 'id': '3', 'uvw': 'UVW_ROT', 'weight': 'WEIGHT', 'imaging_weight': 'WEIGHT'}
Setting default data_group_out  to  {'data': 'DATA_ROT', 'flag': 'FLAG', 'id': '4', 'uvw': 'UVW_ROT', 'weight': 'WEIGHT', 'imaging_weight': 'WEIGHT'}
Setting default chan_tolerance_factor  to  0.005
Setting default oversampling  to  [10, 10]
Setting default max_support  to  [15, 15]
Setting default support_cut_level  to  0.025
Setting default a_chan_num_chunk  to  3
Setting default image_center  to  [500 360]
#########################  Created graph for make_gridding_convolution_function #########################


## Make Mosaic Primary Beam and Image

[make_mosaic_pb](https://cngi-prototype.readthedocs.io/en/latest/_api/autoapi/ngcasa/imaging/make_mosaic_pb/index.html#ngcasa.imaging.make_mosaic_pb.make_mosaic_pb)

[make_image_with_gcf](https://cngi-prototype.readthedocs.io/en/latest/_api/autoapi/ngcasa/imaging/make_image_with_gcf/index.html#ngcasa.imaging.make_image_with_gcf.make_image_with_gcf)

In [33]:
from ngcasa.imaging import make_mosaic_pb

vis_sel_parms = {}
vis_sel_parms['xds'] = 'xds0'
vis_sel_parms['data_group_in_id'] = 3

img_sel_parms = {}
img_xds= xr.Dataset()

img_xds = make_mosaic_pb(mxds,gcf_xds,img_xds,vis_sel_parms,img_sel_parms,grid_parms)

###############################################

from ngcasa.imaging import make_image_with_gcf
from cngi.dio import write_zarr

vis_select_parms = {}
vis_select_parms['xds'] = 'xds0'
vis_select_parms['data_group_in_id'] = 3

img_select_parms = {}
img_select_parms['data_group_in_id'] = 1

norm_parms = {}
norm_parms['norm_type'] = 'flat_sky'

img_xds = make_image_with_gcf(mxds,gcf_xds, img_xds, grid_parms, norm_parms, vis_select_parms, img_select_parms)

img_xds = write_zarr(img_xds,'mosaic_img.img.zarr')

######################### Start make_mosaic_pb #########################
Setting default image_center  to  [500 360]
Setting data_group_in  to  {'data': 'DATA_ROT', 'flag': 'FLAG', 'id': '3', 'uvw': 'UVW_ROT', 'weight': 'WEIGHT', 'imaging_weight': 'WEIGHT'}
Setting default data_group_out  to  {'data': 'DATA_ROT', 'flag': 'FLAG', 'id': '4', 'uvw': 'UVW_ROT', 'weight': 'WEIGHT', 'imaging_weight': 'WEIGHT'}
Setting default data_group_in  to  {'id': '1'}
Setting default data_group_out  to  {'id': '1', 'pb': 'PB', 'weight_pb': 'WEIGHT_PB', 'weight_pb_sum_weight': 'WEIGHT_PB_SUM_WEIGHT'}
#########################  Created graph for make_mosaic_pb #########################
######################### Start make_image_with_gcf #########################
Setting default image_center  to  [500 360]
Setting default single_precision  to  True
Setting data_group_in  to  {'data': 'DATA_ROT', 'flag': 'FLAG', 'id': '3', 'uvw': 'UVW_ROT', 'weight': 'WEIGHT', 'imaging_weight': 'WEIGHT'}
Setting default dat

## Compare CASA and ngCASA Primary Beams

In [61]:
import matplotlib.pylab as plt
import numpy as np
from ipywidgets import interactive
import scipy
from scipy.signal import decimate
from cngi.image import implot

img_xds = xr.open_zarr('mosaic_img.img.zarr')
casa_img_xds = xr.open_zarr('alma12m_3field_dovpTrue_gridder_mosaic.img.zarr')
pb_limit = 0.2


def comparison_plots(chan):
    print('Frequency',img_xds.chan[chan].values/10**9, 'GHz')
    mosaic_pb = img_xds.PB[:,:,chan,0]
    mosaic_pb = mosaic_pb.where(mosaic_pb > pb_limit,other=np.nan)
    
    casa_mosaic_pb = casa_img_xds.PB[:,:,0,chan,0]
    casa_mosaic_pb = casa_mosaic_pb.where(casa_mosaic_pb > pb_limit,other=np.nan)
    
    fig0, ax0 = plt.subplots(1, 2, sharey=True)
    im0 = ax0[0].imshow(mosaic_pb,cmap='jet')
    im1 = ax0[1].imshow(casa_mosaic_pb,cmap='jet')
    ax0[0].title.set_text('ngCASA Mosaic PB')
    ax0[1].title.set_text('CASA Mosaic PB')
    fig0.colorbar(im0, ax=ax0[0], fraction=0.046, pad=0.04)
    fig0.colorbar(im1, ax=ax0[1], fraction=0.046, pad=0.04)
    
    plt.figure()
    plt.plot(mosaic_pb[:,360],label='ngCASA PB')
    plt.plot(casa_mosaic_pb[:,360],'*',label='CASA PB',markersize=0.5)
    plt.legend()
    plt.title('PB cross section')
    
    
    diff_image = mosaic_pb - casa_mosaic_pb
    print(diff_image)
    
    plt.figure()
    #plt.imshow(mosaic_pb - casa_mosaic_pb)
    #diff_image.plot.pcolormesh(ax=['l','m'])
    xr.plot.imshow(diff_image,cmap='jet',cbar_kwargs={"label": ""})
    
    
    #plt.colorbar()
    plt.title('Difference Mosaic PB')
    
    plt.show()

interactive_plot = interactive(comparison_plots, chan=(0, 2))
output = interactive_plot.children[-1]
output.layout.height = '1050px'
interactive_plot


interactive(children=(IntSlider(value=1, description='chan', max=2), Output(layout=Layout(height='1050px'))), …

## Get Simulated Sources l,m Coordinates

In [40]:
from astropy.coordinates import SkyCoord
from astropy.wcs import WCS
rad_to_deg =  180/np.pi
deg_to_rad = np.pi/180
arcsec_to_deg = 1/3600
arcsec_to_rad = np.pi/(180*3600)

phase_center = grid_parms['phase_center'] 
w = WCS(naxis=2)
w.wcs.crpix = np.array(grid_parms['image_size'])//2
w.wcs.cdelt = np.array([-0.04,0.04])*arcsec_to_deg
w.wcs.crval = phase_center*rad_to_deg
w.wcs.ctype = ['RA---SIN','DEC--SIN']

ra = ['12h01m51.903005s','12h01m52.430856s','12h01m52.958707s','12h01m52.259s','12h01m52s','12h01m53.153s']
dec = ['-18d51m49.94373s','-18d51m49.94369s','-18d51m49.94365s','-18d51m42.983s','-18d51m46s','-18d51m59.305s']
ps_skycoord = SkyCoord(ra=ra,dec=dec,frame='fk5')


ra_dec = np.array([ps_skycoord.ra.degree,ps_skycoord.dec.degree]).T
lm_pix_pos = w.all_world2pix(ra_dec, 1)

cell_size = np.array(grid_parms['cell_size'])*arcsec_to_rad
cell_size[0] = -cell_size[0]
image_center = np.array(grid_parms['image_size'])//2
source_lm_pos = lm_pix_pos*cell_size - image_center*cell_size


## Compare CASA and ngCASA Sky Images

In [62]:
import matplotlib.pylab as plt
import numpy as np
from ipywidgets import interactive
import scipy
from scipy.signal import decimate

casa_img_dataset = 0
img_dataset = 0

casa_img_dataset = xr.open_zarr('alma12m_3field_dovpTrue_gridder_mosaic.img.zarr')
img_dataset = xr.open_zarr('my_img.img.zarr')
pb_limit = 0.2

ngcasa_image_name = 'IMAGE'
casa_image_name = 'IMAGE_PBCOR'

def comparison_plots(chan):
    print('Frequency',img_dataset.chan[chan].values/10**9, 'GHz')
    mosaic_pb = img_dataset.PB[:,:,chan,0]
    casa_mosaic_pb = casa_img_dataset.PB[:,:,0,chan,0]
    
    mosaic_img = img_dataset[ngcasa_image_name][:,:,0,chan,0]
    mosaic_img = mosaic_img.where(mosaic_pb > pb_limit,other=np.nan)
    
    casa_mosaic_img = casa_img_dataset[casa_image_name][:,:,0,chan,0]
    casa_mosaic_img = casa_mosaic_img.where(casa_mosaic_pb > pb_limit,other=np.nan)
    
    sim_sources = np.array([1.5,1.76,2.0,2.0,1.456,1.888])
     
    print('################## Flux of Point Sources ##################')
    print('Sim    ','ngCASA  ', 'CASA')
    for i,s in enumerate(sim_sources):
        ngcasa_recovered_val = img_dataset[ngcasa_image_name].isel(time=0,chan=chan,pol=0).interp(l=source_lm_pos[i,0],m=source_lm_pos[i,1]).values
        casa_recovered_val = casa_mosaic_img.interp(l=source_lm_pos[i,0],m=source_lm_pos[i,1]).values      
        print('{0:.3f}  '.format(s),'{0:.4f}  '.format(ngcasa_recovered_val),'{0:.4f}'.format(casa_recovered_val))
    

    print('############ Percentage Difference Flux to Sim ############')
    print('ngCASA  ', 'CASA')
    for i,s in enumerate(sim_sources):
        ngcasa_recovered_val = img_dataset[ngcasa_image_name].isel(time=0,chan=chan,pol=0).interp(l=source_lm_pos[i,0],m=source_lm_pos[i,1]).values
        casa_recovered_val = casa_mosaic_img.interp(l=source_lm_pos[i,0],m=source_lm_pos[i,1]).values      
        print('{0:.4f}  '.format(100*(s-ngcasa_recovered_val)/s),'{0:.4f}'.format(100*(s-casa_recovered_val)/s))

    fig0, ax0 = plt.subplots(1, 2, sharey=True)
    im0 = ax0[0].imshow(mosaic_img,cmap='jet')
    im1 = ax0[1].imshow(casa_mosaic_img,cmap='jet')
    ax0[0].title.set_text('ngCASA Mosaic PB')
    ax0[1].title.set_text('CASA Mosaic PB')
    fig0.colorbar(im0, ax=ax0[0], fraction=0.046, pad=0.04)
    fig0.colorbar(im1, ax=ax0[1], fraction=0.046, pad=0.04)
    
    plt.figure()
    plt.imshow((100*(mosaic_img - casa_mosaic_img)/2),cmap='jet')
    plt.colorbar()
    plt.title('Difference Mosaic Image')
    
    plt.show()

interactive_plot = interactive(comparison_plots, chan=(0, 2))
output = interactive_plot.children[-1]
output.layout.height = '850px'
interactive_plot


interactive(children=(IntSlider(value=1, description='chan', max=2), Output(layout=Layout(height='850px'))), _…